In [ ]:
import gym
from IPython.display import clear_output
from time import sleep
import numpy as np
from matplotlib import pyplot as pl

In [ ]:
env = gym.make("Taxi-v3", render_mode="ansi").env
env.reset()
env.render();

In [ ]:
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'][0])
        print(f'Timestep: {i + 1}')
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)

In [ ]:
actions_dict = {0:'south',
                1:'north',
                2:'east',
                3:'west',
                4:'pickup',
                5:'dropoff'}

In [ ]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])

q_table.shape

In [ ]:
%%time

print('Training the agent')

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False

    while not done:
        if np.random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state,:]) # Exploit learned values

        next_state, reward, done, info = env.step(action)

        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state,:])

        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1

    if i % 100 == 0:
        clear_output(wait=True)
        print(f'Episode: {i}')

print('Training finished.\n')
print(f'Penalties: {penalties}')

In [ ]:
# Find the index of the best action for each state
best_actions = np.argmax(q_table, axis=1)

# Create a new array representing the best actions for each state
best_actions_array = np.zeros_like(q_table)
best_actions_array[np.arange(len(best_actions)), best_actions] = 1

# Plot the Q-table
pl.figure(figsize=(16, 8))

# Plot Q-table
pl.subplot(2, 1, 1)
pl.imshow(q_table.T, interpolation='nearest', aspect=10)
pl.yticks(range(len(actions_dict)), [actions_dict[k] for k in range(len(actions_dict))])
pl.title('Q table')
pl.xlabel('States')
pl.ylabel('Actions')

# Plot best actions
pl.subplot(2, 1, 2)
pl.imshow(best_actions_array.T, interpolation='nearest', aspect=10)
pl.yticks(range(len(actions_dict)), [actions_dict[k] for k in range(len(actions_dict))])
pl.title('Best Actions')
pl.xlabel('States')
pl.ylabel('Actions')

pl.tight_layout()
pl.show()


In [ ]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

print(q_table[state])
print(f'Selected action: {actions_dict[np.argmax(q_table[state])]}')

In [ ]:
print("Evaluate agent's performance after Q-learning")

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0

    done = False

    while not done:
        action = np.argmax(q_table[state,:])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f'Results after {episodes} episodes:')
print(f'Average timesteps per episode: {total_epochs / episodes}')
print(f'Average penalties per episode: {total_penalties / episodes}')


In [ ]:
print("Visualizing an individual episode")

state = env.reset()
epochs, penalties, reward = 0, 0, 0

done = False

while not done:
    action = np.argmax(q_table[state,:])
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1

    epochs += 1

    clear_output(wait=True)
    env.render()
    sleep(.2)

print(f'Timesteps: {epochs}')
print(f'Penalties: {penalties}')